# Transpose file

In [ ]:
from pathlib import Path
import pandas as pd

import vaep

import config

Paramters

In [ ]:
out_folder = Path('data/selected/peptides') 
fname = out_folder / 'intensities_wide_selected_N42881_M07441.pkl'

In [ ]:
def get_template(fname, split='_N'):
    ext = fname.suffix
    stem = fname.stem.split(split)[0]
    return f"{stem}{{}}{ext}"

def memory_usage_in_mb(df):
    return df.memory_usage(deep=True).sum() / (2**20)

template = get_template(fname)
template

In [ ]:
files_out = {}

In [ ]:
%%time
df = pd.read_pickle(fname)
df.head()

In [ ]:
df.memory_usage(deep=True).sum() / (2**20)

Here reading the csv file is slightly faster and consumes less memory.

- dtype: `float64` -> missing values as `np.nan`
- but: saving to csv will be larger.

In [ ]:
# %%time
# df = pd.read_csv(fname.with_suffix('.csv'), index_col=0)
# df.memory_usage(deep=True).sum() / (2**20) 

In [ ]:
%%time
count_samples = df.notna().sum()

fname = out_folder / 'count_samples.json'
count_samples.to_json(fname)

vaep.plotting.make_large_descriptors(size='medium')

ax = count_samples.sort_values().plot(rot=90, ylabel='observations')
vaep.savefig(ax.get_figure(), fname)

In [ ]:
%%time
df = df.T
df.memory_usage(deep=True).sum() / (2**20)

In [ ]:
%%time
fname = out_folder / config.insert_shape(df, template=template)
files_out[fname.name] = fname.as_posix()
df.to_pickle(fname)

In [ ]:
%%time
fname = fname.with_suffix('.csv')
files_out[fname.name] = fname.as_posix()
df.to_csv(fname, chunksize=1_000)

In [ ]:
count_features = df.notna().sum()
fname = out_folder / 'count_feat.json'
count_features.to_json(fname)

ax = count_features.sort_values().plot(rot=90, ylabel='observations') 
vaep.savefig(ax.get_figure(), fname)

## Present abesent pattern

In [ ]:
df = df.notna().astype(pd.Int8Dtype())
df

In [ ]:
%%time
fname = out_folder / config.insert_shape(df,  'absent_0_present_1_selected{}.pkl')

files_out[fname.name] = fname.as_posix()
df.to_pickle(fname)

In [ ]:
files_outfname = fname.with_suffix('.csv')
files_out[fname.name] = fname.as_posix()
df.replace(0, pd.NA).to_csv(fname.with_suffix('.csv'), chunksize=1_000)

## Files written

In [ ]:
files_out